In [ ]:
"""
Visualization of speaker statistics

"""

In [ ]:
import re, itertools
import networkx as nx

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/katiehuang/Desktop/metis/projects/onl_ds5_project_4/py')
from word_cloud import *
import importlib

In [ ]:
speaker_info = pd.read_pickle('../dump/speaker_info_allp')

In [ ]:
# Drop speaker if there's missing values
speaker_info_cut = speaker_info.dropna()
speaker_info_cut.shape

In [ ]:
# List speaker professions
professions = speaker_info_cut.profession.tolist()
professions = [profession.lower() for profession in professions]
len(professions),
# professions

In [ ]:
# Categorize speaker professions

category =[]

publish = ['novelist','writer','author','cartoonist','poet','journalist',
           'illustrator','comic','memoirist']
med_law = ['physician','surgeon','psychologist','neurosurgeon','lawyer','gynecologist','judge']
entertainment = ['humorist','filmmaker','actress','actor','host','comedian',
                 'director','songwriter','commentator','screenwriter','producer',
                 'podcaster','personality','designer','broadcaster',
                 'playwright','animator',]
art = ['artist','musician','composer','trumpeter','mezzo-soprano']
sport = ['player','soccer','basketball','sportscaster','baseball','coach','skier',
         'nfl','sports announcer','rower','tennis','football']
tech_business = ['entrepreneur','executive','chief','ceo','senior vice president',
                 'businesswoman','businessman','business','boeing','philanthropist']
science = ['astrophysicist','astronaut','biologist','scientist','science']
education_research = ['researcher','economist','educator','university',
                      'environmentalist','philosopher','professor']
politics = ['police','senator','businessperson','first lady','united states',
            'admiral','presidential','governor','political','mayor','legislative',
            'u.s. president','navy','officer','official','president of ireland',]
other = ['spiritual']

categories = [publish, publish, med_law ,entertainment, sport, tech_business,
              science, politics, education_research, other]

for profession in professions:
    if any(x in profession for x in publish):
        category.append('Publish')
    elif any(x in profession for x in med_law):
        category.append('Med/Law')
    elif any(x in profession for x in entertainment):
        category.append('Entertainment')
    elif any(x in profession for x in art):
        category.append('Art')
    elif any(x in profession for x in sport):
        category.append('Sport')
    elif any(x in profession for x in tech_business):
        category.append('Tech/Business')
    elif any(x in profession for x in science):
        category.append('Science')
    elif any(x in profession for x in politics):
        category.append('Politics')
    elif any(x in profession for x in education_research):
        category.append('Education/Research')
    else:
        category.append('Other')

In [ ]:
len(category)

In [ ]:
speaker_info_cut['category'] = category

In [ ]:
speaker_info_cut

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]
speaker_info_cut.category.value_counts().plot(kind='pie');

In [ ]:
speaker_info_cut[speaker_info_cut.category == "Other"]

In [ ]:
# Pickle
speaker_info_cut.to_pickle('../dump/speaker_info_cut')